In [1]:
import pandas as pd
import numpy as np
import torch

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\abina\anaconda3\envs\cuda_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
movies = pd.read_csv('movies_data.csv')
ratings = pd.read_csv('ratings_data.csv')

In [3]:

model = SentenceTransformer('all-MiniLM-L6-v2') 
def get_embeddings(text):
    return model.encode(text) 

c:\Users\abina\anaconda3\envs\cuda_env\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
movies['genra_crew'] = movies['genres'].fillna('NA') + '. Director: ' + movies['director'].fillna('NA') + '. Cast: ' + movies['cast'].fillna('NA')
movies['overview'] = movies['overview'].fillna('NA')

In [5]:
genra_crew_embeddings = np.apply_along_axis(get_embeddings, axis = 0,arr = movies['genra_crew'].values)
movie_overview_embeddings = np.apply_along_axis(get_embeddings, axis = 0,arr = movies['overview'].values)

In [6]:
overall_embedding = genra_crew_embeddings + movie_overview_embeddings

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

cos_sim  = cosine_similarity(overall_embedding)

In [8]:
#np.save('parameters/content_based_sims.npy', cos_sim)

In [13]:
def recommend(movie, df):
    idx = df[df['title']==movie].index[0]
    
    top = sorted(list(enumerate(cos_sim[idx])), key= lambda x: -x[1])[1:10]
    top_movie_title = [movie[0] for movie in top]

    
    reco = df[df['id'].isin(top_movie_title)]
    reco = reco[reco['vote_count']>50].sort_values(by = 'vote_average', ascending = False)
    reco = reco['title'].tolist()
    

    return reco

In [14]:
m = 'Star Wars: Episode I - The Phantom Menace 1999'

In [15]:
recommend(m, movies)

['The Empire Strikes Back 1980',
 'Star Wars 1977',
 'Return of the Jedi 1983',
 'Indiana Jones and the Temple of Doom 1984',
 'Star Wars: Episode III - Revenge of the Sith 2005',
 'Star Wars: Episode II - Attack of the Clones 2002',
 'Star Wars: The Clone Wars 2008',
 'Ben-Hur 2016']

In [12]:
from fuzzywuzzy import process

def get_top_similar_strings(target, string_list, top_n=5):
    top_matches = process.extract(target, string_list, limit=top_n)
    return top_matches

get_top_similar_strings('star wars', movies.title.tolist())

c:\Users\abina\anaconda3\envs\cuda_env\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


[('Star Wars 1977', 90),
 ('Star Wars: Episode I - The Phantom Menace 1999', 90),
 ('Star Wars: Episode II - Attack of the Clones 2002', 90),
 ('Star Wars: Episode III - Revenge of the Sith 2005', 90),
 ('Star Wars: The Clone Wars 2008', 90)]